# 우선순위 회원 미들웨어가 적용된 호텔 예약

이 노트북은 Microsoft Agent Framework를 사용한 **함수 기반 미들웨어**를 보여줍니다. 조건부 워크플로우 예제를 확장하여 우선순위 회원에게 특별 권한을 부여하는 미들웨어 계층을 추가합니다.

## 배우게 될 내용:
1. **함수 기반 미들웨어**: 함수 결과 가로채기 및 수정
2. **컨텍스트 접근**: 실행 후 `context.result` 읽기 및 수정
3. **비즈니스 로직 구현**: 우선순위 회원 혜택
4. **결과 재정의**: 사용자 상태에 따른 함수 결과 변경
5. **동일한 워크플로우, 다른 결과**: 미들웨어에 의한 동작 변화

## 미들웨어와 함께하는 워크플로우 아키텍처:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## 조건부 워크플로우와의 주요 차이점:

**미들웨어 없이** (14-conditional-workflow.ipynb):
- 파리에는 방이 없음 → alternative_agent로 라우팅

**미들웨어 포함** (이 노트북):
- 일반 사용자 + 파리 → 방 없음 → alternative_agent로 라우팅
- 우선순위 사용자 + 파리 → 🌟 미들웨어가 재정의! → 가능 → booking_agent로 라우팅

## 필요한 사항:
- Microsoft Agent Framework 설치
- 조건부 워크플로우 이해 (14-conditional-workflow.ipynb 참조)
- GitHub 토큰 또는 OpenAI API 키
- 미들웨어 패턴 기본 이해


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## 1단계: 구조화된 출력을 위한 Pydantic 모델 정의

이 모델들은 에이전트가 반환할 **스키마**를 정의합니다. 미들웨어가 가용성 결과를 수정할 때 이를 추적하기 위해 `priority_override` 필드를 추가했습니다.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: 우선 멤버 데이터베이스 정의

이 데모에서는 우선 멤버십 데이터베이스를 시뮬레이션합니다. 실제 환경에서는 실제 데이터베이스나 API를 쿼리하게 됩니다.

**우선 멤버:**
- `alice@example.com` - VIP 멤버
- `bob@example.com` - 프리미엄 멤버  
- `priority_user` - 테스트 계정


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Step 3: 호텔 예약 도구 만들기

조건부 워크플로우와 동일하지만, 이제 미들웨어에 의해 가로채집니다!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 우선순위 체크 미들웨어 만들기 (핵심 기능!)

이것이 이 노트북의 **핵심 기능**입니다. 미들웨어는:

1. 호텔 예약 함수 호출을 **가로챕니다**
2. `next(context)`를 호출하여 함수를 정상적으로 **실행합니다**
3. `context.result`에서 결과를 **검사합니다**
4. 사용자가 우선순위이고 예약 가능한 방이 없으면 결과를 **재정의합니다**
5. 수정된 결과를 에이전트에 다시 **반환합니다**

**핵심 패턴:**
```python
async def my_middleware(context, next):
    await next(context)  # 함수 실행
    # 이제 context.result에 함수의 출력이 포함됩니다
    if some_condition:
        context.result = new_value  # 재정의!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## 5단계: 라우팅을 위한 조건 함수 정의

조건부 워크플로우와 동일한 조건 함수 - 라우팅을 결정하기 위해 구조화된 출력을 검사합니다.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Step 6: 사용자 지정 표시 실행기 만들기

이전과 동일한 실행기 - 최종 워크플로 출력물을 표시합니다.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Step 7: 환경 변수 로드

LLM 클라이언트(GitHub Models 또는 OpenAI)를 구성합니다.


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## 8단계: 미들웨어를 사용하여 AI 에이전트 생성

**주요 차이점:** availability_agent를 생성할 때, `middleware` 매개변수를 전달합니다!

이것이 priority_check_middleware를 에이전트의 함수 호출 파이프라인에 주입하는 방법입니다.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Step 9: 워크플로 빌드

이전과 동일한 워크플로 구조 - 가용성에 따른 조건부 라우팅.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: 테스트 케이스 1 - 파리에 있는 일반 사용자 (재정의 없음)

일반 사용자가 파리 예약 시도 → 객실 없음 → alternative_agent로 경로 지정


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Step 11: Test Case 2 - 🌟 파리 우선 사용자 (오버라이드 포함!)

우선 멤버가 파리 예약 시도 → 처음에는 방 없음 → 🌟 미들웨어가 오버라이드! → booking_agent로 라우팅

**이것이 미들웨어의 강력함을 보여주는 핵심 데모입니다!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Step 12: 테스트 케이스 3 - 스톡홀름 우선 사용자 (이미 사용 가능)

우선 사용자가 스톡홀름 시도 → 객실 사용 가능 → 재정의 필요 없음 → booking_agent로 라우팅

이는 미들웨어가 필요할 때만 작동한다는 것을 보여줍니다!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## 주요 내용과 미들웨어 개념

### ✅ 배운 내용:

#### **1. 함수 기반 미들웨어 패턴**

미들웨어는 간단한 비동기 함수로 함수 호출을 가로챕니다:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # 함수 실행 전
    print("Intercepting...")
    
    # 함수를 실행합니다
    await next(context)
    
    # 함수 실행 후 - 결과를 확인합니다
    if context.result:
        # 필요한 경우 결과를 수정합니다
        context.result = modified_value
```

#### **2. 컨텍스트 접근과 결과 오버라이드**

- `context.function` - 호출된 함수에 접근
- `context.arguments` - 함수 인자 읽기
- `context.kwargs` - 추가 매개변수 접근
- `await next(context)` - 함수 실행
- `context.result` - 함수 결과 읽기/수정

#### **3. 비즈니스 로직 구현**

우리 미들웨어는 우선순위 회원 혜택을 구현합니다:
- **일반 사용자**: 변경 없음, 표준 워크플로우
- **우선순위 사용자**: "사용 불가" → "사용 가능" 오버라이드
- **조건부 로직**: 필요할 때만 오버라이드

#### **4. 같은 워크플로우, 다른 결과**

미들웨어의 강점:
- ✅ 워크플로우 구조 변경 없음
- ✅ 툴 함수 변경 없음
- ✅ 조건부 라우팅 로직 변경 없음
- ✅ 미들웨어만 → 다른 행동!

### 🚀 실제 응용 사례:

1. **VIP/프리미엄 기능**
   - 프리미엄 사용자에 대한 속도 제한 오버라이드
   - 자원 우선 접근 제공
   - 프리미엄 기능 동적 잠금 해제

2. **A/B 테스트**
   - 사용자를 다른 구현으로 라우팅
   - 특정 사용자와 새 기능 테스트
   - 점진적 기능 롤아웃

3. **보안 및 규정 준수**
   - 함수 호출 감사
   - 민감한 작업 차단
   - 비즈니스 규칙 시행

4. **성능 최적화**
   - 특정 사용자에 대한 결과 캐싱
   - 가능한 경우 비용이 큰 작업 건너뛰기
   - 동적 자원 할당

5. **오류 처리 및 재시도**
   - 오류 잡기 및 우아한 처리
   - 재시도 로직 구현
   - 대체 구현으로 폴백

6. **로깅 및 모니터링**
   - 함수 실행 시간 추적
   - 매개변수와 결과 로깅
   - 사용 패턴 모니터링

### 🔑 데코레이터와의 주요 차이점:

| 기능 | 데코레이터 | 미들웨어 |
|---------|-----------|------------|
| **범위** | 단일 함수 | 에이전트 내 모든 함수 |
| **유연성** | 정의 시 고정 | 런타임에 동적 |
| **컨텍스트** | 제한적 | 에이전트 전체 컨텍스트 |
| **구성** | 다중 데코레이터 | 미들웨어 파이프라인 |
| **에이전트 인식** | 아니오 | 예 (에이전트 상태 접근 가능) |

### 📚 미들웨어 사용 시기:

✅ **미들웨어를 사용할 때:**
- 사용자/세션 상태에 따라 동작을 변경해야 할 때
- 여러 함수에 로직을 적용하고 싶을 때
- 에이전트 수준 컨텍스트에 접근해야 할 때
- 횡단 관심사(로깅, 인증 등)를 구현할 때

❌ **미들웨어를 사용하지 말아야 할 때:**
- 간단한 입력 검증(파이댄틱 사용)
- 함수별 고유 로직(함수 내에 유지)
- 일회성 변경(함수 자체를 변경)

### 🎓 고급 패턴:

```python
# 여러 미들웨어 (실행 순서가 중요함!)
middleware=[
    logging_middleware,      # 먼저 로그 기록
    auth_middleware,         # 그 다음 인증 확인
    cache_middleware,        # 그 다음 캐시 확인
    rate_limit_middleware,   # 그 다음 속도 제한
    priority_check_middleware  # 마지막으로 우선순위 확인
]

# 조건부 미들웨어 실행
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # 결과 수정
    else:
        # 실행 완전히 건너뛰기
        context.result = cached_value
```

### 🔗 관련 개념:

- **에이전트 미들웨어**: agent.run() 호출 가로채기
- **함수 미들웨어**: 툴 함수 호출 가로채기(우리가 사용한 것!)
- **미들웨어 파이프라인**: 순서대로 실행되는 미들웨어 체인
- **컨텍스트 전파**: 미들웨어 체인을 통한 상태 전달


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**면책 조항**:
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 노력하였으나, 자동 번역에는 오류나 부정확한 부분이 있을 수 있음을 유의하시기 바랍니다. 원문은 해당 언어의 원본 문서를 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 본 번역 사용으로 인한 어떠한 오해나 오해의 소지에 대해 당사는 책임을 지지 않습니다.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
